In [14]:
from pymongo import MongoClient

# Configuração do banco de dados
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "pokedex"
COLLECTION_NAME = "pokemons"

# Conectando ao MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Deletando todos os documentos da collection
result = collection.delete_many({})

# Informando o resultado
print(f"{result.deleted_count} documentos foram deletados.")


0 documentos foram deletados.


In [15]:
import requests
import pymongo
import time

# URL do endpoint da PokeAPI
url = "https://pokeapi.co/api/v2/pokemon/?offset=0&limit=2"

# Conectando com o MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["pokedex"]
collection = db["pokemons"]

# Fazendo a requisição HTTP
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Convertendo a resposta para JSON
    data = response.json()

    # Obtendo a lista de pokémons
    pokemons = data["results"]

    for index, pokemon in enumerate(pokemons):
        pokemon_url = pokemon["url"]
        print(f"Obtendo dados para o Pokémon {index + 1}/{len(pokemons)}: {pokemon['name']}")

        pokemon_data = requests.get(pokemon_url).json()

        # Estruturando os dados para inserção no MongoDB
        record = {
            "name": pokemon_data["name"],
            "url": pokemon_url,
            "abilities": [ability["ability"]["name"] for ability in pokemon_data["abilities"]],
            "base_experience": pokemon_data["base_experience"],
            "forms": pokemon_data["forms"],
            "game_indices": pokemon_data["game_indices"],
            "height": pokemon_data["height"],
            "held_items": pokemon_data["held_items"],
            "id": pokemon_data["id"],
            "is_default": pokemon_data["is_default"],
            "location_area_encounters": pokemon_data["location_area_encounters"],
            "moves": pokemon_data["moves"],
            "n_order": pokemon_data["order"],
            "past_types": pokemon_data["past_types"],
            "species": pokemon_data["species"],
            "sprites": pokemon_data["sprites"],
            "stats": pokemon_data["stats"],
            "types": pokemon_data["types"],
            "weight": pokemon_data["weight"]
        }

        # Inserindo o registro no MongoDB
        collection.insert_one(record)

        # Aguarde 1 segundo antes de fazer a próxima requisição
        time.sleep(0.3)

    print("Pokémons inseridos com sucesso no MongoDB!")
else:
    print(f"Erro ao acessar a API da PokeAPI. Código de status: {response.status_code}")


Obtendo dados para o Pokémon 1/2: bulbasaur
Obtendo dados para o Pokémon 2/2: ivysaur
Pokémons inseridos com sucesso no MongoDB!


In [16]:
import pymongo
import pika
import json

# Configurações do MongoDB
MONGO_URI = "mongodb://localhost:27017/"
client = pymongo.MongoClient(MONGO_URI)
db = client["pokedex"]
collection = db["pokemons"]

# Configuração de conexão com o RabbitMQ
credentials = pika.PlainCredentials('user', 'password')
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost', 5672, '/', credentials))
channel = connection.channel()

# Garantindo que a fila exista (se não existir, ela será criada)
channel.queue_declare(queue='pokemon_queue', durable=True)

# Lendo os registros do MongoDB
records = collection.find()

for record in records:
    # O ID do MongoDB não é serializável, então vamos removê-lo
    record['_id'] = str(record['_id'])

    # Enviando o registro para o RabbitMQ
    channel.basic_publish(exchange='',
                          routing_key='pokemon_queue',
                          body=json.dumps(record),
                          properties=pika.BasicProperties(
                              delivery_mode=2,  # torna a mensagem persistente
                          ))

print("Registros do MongoDB enviados para a fila do RabbitMQ!")
connection.close()


Registros do MongoDB enviados para a fila do RabbitMQ!
